In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
!pip install beautifulsoup4 --no-cache

In [ ]:
!pip install html5lib

In [147]:
!pip install chardet

Looking in indexes: https://artifactory.tools-k8s.hellofresh.io/artifactory/api/pypi/py/simple


In [345]:
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import base64
import email
from apiclient import errors
from bs4 import BeautifulSoup

In [346]:
SCOPES = 'https://www.googleapis.com/auth/gmail.readonly'

creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

In [347]:
service = build('gmail', 'v1', credentials=creds)

# Call the Gmail API
results = service.users().labels().list(userId='me').execute()
labels = results.get('labels', [])

In [348]:
def list_threads_matching_query(service, user_id='me', query=''):
    try:
        response = service.users().threads().list(userId=user_id, q=query).execute()
        threads = []
        if 'threads' in response:
            threads.extend(response['threads'])
        while 'nextPageToken' in response:
            page_token = response['nextPageToken']
            response = service.users().threads().list(userId=user_id, q=query,
                                            pageToken=page_token).execute()
            threads.extend(response['threads'])
        return threads
    except errors.HttpError:
        print(f'An error occurred: {error}')
        
def get_thread(service, thread_id, user_id='me'):
    try:
        thread = service.users().threads().get(userId=user_id, id=thread_id).execute()
        messages = thread['messages']
        return thread
    except errors.HttpError:
        print(f'An error occurred: {error}')
        
def get_message(service, msg_id, format, user_id='me'):
    try:
        message = service.users().messages().get(userId=user_id, id=msg_id, format=format).execute()
        return message
    except errors.HttpError:
        print(f'An error occurred: {error}')

In [349]:
users = ['cederberg.erik@gmail.com', 'steen.eriksteen@gmail.com',  'valdemar@scandikitchen.co.uk', 'johannes.siven@gmail.com', 'johannessiven@gmail.com', 'max.y.leander@gmail.com', 'macki.leander@gmail.com', 'erik.cederberg@kontigo.se']
user_sets = [
    {'cederberg.erik@gmail.com', 'steen.eriksteen@gmail.com',  'valdemar@scandikitchen.co.uk', 'johannes.siven@gmail.com', 'max.y.leander@gmail.com'},
    {'erik.cederberg@kontigo.se', 'steen.eriksteen@gmail.com',  'valdemar@scandikitchen.co.uk', 'johannes.siven@gmail.com', 'max.y.leander@gmail.com'},
    {'cederberg.erik@gmail.com', 'steen.eriksteen@gmail.com',  'valdemar@scandikitchen.co.uk', 'johannessiven@gmail.com', 'max.y.leander@gmail.com'},
    {'erik.cederberg@kontigo.se', 'steen.eriksteen@gmail.com',  'valdemar@scandikitchen.co.uk', 'johannessiven@gmail.com', 'max.y.leander@gmail.com'},
    {'cederberg.erik@gmail.com', 'steen.eriksteen@gmail.com',  'valdemar@scandikitchen.co.uk', 'johannes.siven@gmail.com', 'macki.leander@gmail.com'},
    {'erik.cederberg@kontigo.se', 'steen.eriksteen@gmail.com',  'valdemar@scandikitchen.co.uk', 'johannes.siven@gmail.com', 'macki.leander@gmail.com'},
    {'cederberg.erik@gmail.com', 'steen.eriksteen@gmail.com',  'valdemar@scandikitchen.co.uk', 'johannessiven@gmail.com', 'macki.leander@gmail.com'},
    {'erik.cederberg@kontigo.se', 'steen.eriksteen@gmail.com',  'valdemar@scandikitchen.co.uk', 'johannessiven@gmail.com', 'macki.leander@gmail.com'},
]
receiver_sets = []
for user_set in user_sets:
    for user in users:
        try:
            receiver_sets.append(user_set.difference({user}))
        except KeyError:
            pass

In [350]:
additional_receiver_sets = []
for user_set in receiver_sets:
    additional_receiver_sets.append(user_set.difference({'valdemar@scandikitchen.co.uk'}))
receiver_sets.extend(additional_receiver_sets)

In [351]:
threads = set()
for user_set in receiver_sets:
    new_threads = list_threads_matching_query(service, query=' '.join([f'to:{user}' for user in user_set]))
    threads.update([t['id'] for t in new_threads])
threads = list(threads)

In [352]:
len(threads)

292

In [353]:
import re
import quopri
import chardet

def get_msg(lines, sender, encoding):
    text = sender + '\n' + '\n'.join(lines)
    try:
        text = quopri.decodestring(text)
    except ValueError:
        pass
    try:
        text = text.decode(encoding)
    except LookupError:
        text = text.decode('utf-8')
    except AttributeError:
        pass
    return text

In [357]:
base64_str = 'Content-Transfer-Encoding: base64'

def parse_message(msg_id):
    encoding = 'utf-8'
    sender = [h for h in get_message(service, msg_id, 'full')['payload']['headers'] if h['name'].lower() == 'from'][0]['value']
    msg = get_message(service, msg_id, 'raw')
    html = base64.urlsafe_b64decode(msg['raw'])
    soup = BeautifulSoup(html, 'html5lib')
    text = str(email.message_from_string(soup.getText()).get_payload()[0])
    if base64_str in text:
        text = text[text.index(base64_str)+len(base64_str):]
        text = base64.urlsafe_b64decode(text)
        try:
            text = text.decode('utf-8')
        except UnicodeDecodeError:
            return None
    lines = []
    for line in text.split('\n'):
        if line.startswith('>') or line.startswith('--=20') \
        or re.compile('^On [A-Z]\w\w, [A-Z]\w\w \d?\d, 20\d\d(,| at) \d?\d:\d\d').match(line) \
        or re.compile('^\w\w\w \d?\d \w\w\w. 20\d\d kl. \d\d:\d\d skrev').match(line) \
        or re.compile('^Den \w\w\w \d?\d \w\w\w. 20\d\d \d?\d:\d\d').match(line) \
        or re.compile('^On \d?\d [A-Z]\w+? 20\d\d at \d?\d:\d\d,').match(line) \
        or re.compile('^Den \d?\d \w+? 20\d\d \d?\d:\d\d skrev').match(line) \
        or re.compile('^\w+? \d?\d \w+? 20\d\d kl. \d?\d:\d\d').match(line) \
        or re.compile('^On \d?\d \w+? 20\d\d \d?\d:\d\d [ap].m.').match(line) \
        or re.compile('^Den \d?\d \w+?\. 20\d\d \d?\d:\d\d skrev').match(line) \
        or re.compile('^\w+? \d?\d \w+?. 20\d\d kl. \d?\d:\d\d skrev').match(line):
            break
        if line.startswith('Content-') or re.compile('^--[a-z0-9]').match(line) or 'charset=' in line:
            if line.startswith('Content-Type'):
                try:
                    encoding = line[line.index('charset=') + 8:]
                except ValueError:
                    pass
            continue
        lines.append(line)
    return get_msg(lines, sender, encoding)

def parse_thread(thread_id):
    with open(f'../threads/{thread_id}.txt', 'w') as f:
        raw_messages = get_thread(service, thread_id)['messages']
        if len(raw_messages) < 2:
            return
        print(f'Parsing thread ID = {thread_id}...', end='')
        parsed_messages = []
        for msg in raw_messages:
            try:
                parsed_msg = parse_message(msg['id'])
                if parsed_msg:
                    parsed_messages.append(parsed_msg)
            except Exception:
                print(f"Could not parse message with ID = {msg['id']}")
        file_string = ''
        for msg in parsed_messages:
            file_string += '*'*50 + '\n'
            file_string += msg + '\n'
        f.write(file_string)
        print('done!')

In [358]:
for t in threads:
    parse_thread(t)

Parsing thread ID = 166c51b119ad5820...done!
Parsing thread ID = 161bf41cdf8bb6a2...done!
Parsing thread ID = 1668cd2bc4860337...done!
Parsing thread ID = 16a5a8dfb6e5224e...done!
Parsing thread ID = 16827f1a06306e70...done!
Parsing thread ID = 1695cad49c1dffeb...done!
Parsing thread ID = 164570103cbf4c9b...done!
Parsing thread ID = 169cbabd9eeb8194...done!
Parsing thread ID = 161bd1b840820296...done!
Parsing thread ID = 167893005ec8302a...done!
Parsing thread ID = 14ce6c63d5916d77...done!
Parsing thread ID = 1452c40203396f19...done!
Parsing thread ID = 166ca38a2440fe9a...Could not parse message with ID = 166e3fe6fd12a1da
done!
Parsing thread ID = 169303bed7ffa862...done!
Parsing thread ID = 1615ad9cc185317a...done!
Parsing thread ID = 16640c5042d617ed...done!
Parsing thread ID = 1697b854172a61d6...done!
Parsing thread ID = 162402de7b88a3b3...done!
Parsing thread ID = 14a25d388acae988...done!
Parsing thread ID = 1665e2d9ecf11356...done!
Parsing thread ID = 16c99f529bed061e...done!
Pars

Parsing thread ID = 1617a4712fb02d61...done!
Parsing thread ID = 1635dd429045b0d7...done!
Parsing thread ID = 16525e3fdd103cb0...done!
Parsing thread ID = 15f8115b79682bb0...done!
Parsing thread ID = 166aeebc776d31ca...done!
Parsing thread ID = 16b46394e8cad0c6...done!
Parsing thread ID = 1653c4596685b914...done!
Parsing thread ID = 16d08528fd1e6949...done!
Parsing thread ID = 163969c42cb0a1bb...Could not parse message with ID = 163978f887eb2cb2
done!
Parsing thread ID = 163d546bbc0e0219...done!
Parsing thread ID = 16ad934d85d7cfc0...done!
Parsing thread ID = 168614a22b178c11...done!
Parsing thread ID = 16166085a03ebce6...done!
Parsing thread ID = 16d5fd24bd5e367c...done!
Parsing thread ID = 16b212da61c39e6a...done!
Parsing thread ID = 16a84bf2d482c9df...done!
Parsing thread ID = 16b750fafd800758...done!
Parsing thread ID = 164cbc51521dc4dd...done!
Parsing thread ID = 160bad2ba5e2beac...done!
Parsing thread ID = 1618bc5c8191255d...done!
Parsing thread ID = 1617edee48179962...done!
Pars

In [369]:
iterator = iter(threads)
big_file = 'cirkeln.txt'

In [909]:
t = next(iterator)
with open(f'../threads/{t}.txt') as f:
    text = ''.join([i if re.compile('[a-öA-Ö0-9,.!:; \n@#$%^&*(){}\[\]\\|"\'/?<>=+-_`~]').match(i) else ' ' for i in f.read()]) + '\n'
    text = text.replace('*'*50, '')
    print(text)

StopIteration: 

In [906]:
with open(f'../threads/{big_file}', 'a') as out:
    out.write(text + '\n')

In [910]:
print(f'{float(threads.index(t) / len(threads)):.2%}')

99.66%
